1. add a section to check if recon-all has already been performed
2. subsegmentations
3. prints
4. different atlases
5. tracula check
6. create report
7. check the list (not to forget sth)
8. USZ MRI running

In [ ]:


########################### STEP 1 ###################################
# 			  recon-all, subcortical segmentation                    #
######################################################################

## convert to nifti format and set Freesurfer paths
dcm2niix -f raw_T1 -o . $RAW_T1
dcm2niix -f raw_T2 -o . $RAW_T2


### very very important for ubuntu
export FREESURFER_HOME=/usr/local/freesurfer/8.0.0
export SUBJECTS_DIR=/home/ubuntu/data/fs_subjects_dir
export LD_LIBRARY_PATH=$FREESURFER_HOME/MCRv97/runtime/glnxa64:$FREESURFER_HOME/MCRv97/bin/glnxa64:$FREESURFER_HOME/MCRv97/sys/os/glnxa64:$FREESURFER_HOME/MCRv97/extern/bin/glnxa64
source $FREESURFER_HOME/SetUpFreeSurfer.sh
export FS_V8_XOPTS=0 && recon-all -all -i I50 -s Subj001
nohup bash -c 'export FS_V8_XOPTS=0 && recon-all -all -i I50 -s Subj001' > output.log 2>&1 &



export FREESURFER_HOME=/Applications/freesurfer/dev
export SUBJECTS_DIR=$FREESURFER_HOME/subjects
source $FREESURFER_HOME/SetUpFreeSurfer.sh

## run surface reconstruction
recon-all -s $SUB_ID -i raw_T1
recon-all -all -subjid $SUB_ID
cd $SUBJECTS_DIR/$SUB_ID
mkdir tables

recon-all -subjid dvob -i raw_anat_T1.nii

## segmentation of hippocampal subfields, nuclei of the amygdala, thalamic nuclei and Brainstem (T1, T1+T2)
segmentHA_T1.sh $SUB_ID $SUBJECTS_DIR 
segmentBS.sh $SUB_ID $SUBJECTS_DIR
segmentHA_T2.sh $SUB_ID raw_T2 T1_T2 1 $SUBJECTS_DIR
segmentThalamicNuclei.sh $SUB_ID $SUBJECTS_DIR
# mri_segment_hypothalamic_subunits
# segmentThalamicNuclei_DTI.sh -s $SUB_ID not available yet

########################### STEP 1.5 #################################
# 			 			 Schaefer Atlas  					         #
######################################################################
## dont forget gcs_file_dir=/Users/payamsadeghishabestari/Antinomics/codes/gcs

for hemi in "lh" "rh"; do
	for N in 100 200 300 400; do
		for net_option in 7 17; do
			mris_ca_label -l $SUBJECTS_DIR/$SUB_ID/label/${hemi}.cortex.label \
			$SUB_ID ${hemi} $SUBJECTS_DIR/$SUB_ID/surf/${hemi}.sphere.reg \
			$gcs_file_dir/${hemi}.Schaefer2018_${N}Parcels_${net_option}Networks.gcs \
			$SUBJECTS_DIR/$SUB_ID/label/${hemi}.Schaefer2018_${N}Parcels_${net_option}Networks_order.annot
		done
	done
done

## put into loop
mri_aparc2aseg --s $SUB_ID --o $SUBJECTS_DIR/$SUB_ID/mri/Sch_${N}Parcels_${net_option}.mgz --annot Schaefer2018_${N}Parcels_${net_option}Networks_order

## just visualize it
freeview -v $SUBJECTS_DIR/$SUB_ID/mri/Sch_${N}Parcels_${net_option}.mgz:colormap=lut:lut=/Users/payamsadeghishabestari/Antinomics/codes/Sch_LUT_files/Schaefer2018_100Parcels_7Networks_order_LUT.txt

## put it into loop

mris_anatomical_stats <subject_name> lh \
  -f $SUBJECTS_DIR/<subject_name>/stats/lh.Schaefer2018_<N>Parcels_<7/17>Networks.stats \
  -b -a $SUBJECTS_DIR/<subject_name>/label/lh.Schaefer2018_<N>Parcels_<7/17>Networks.annot

mris_anatomical_stats <subject_name> rh \
  -f $SUBJECTS_DIR/<subject_name>/stats/rh.Schaefer2018_<N>Parcels_<7/17>Networks.stats \
  -b -a $SUBJECTS_DIR/<subject_name>/label/rh.Schaefer2018_<N>Parcels_<7/17>Networks.annot

########################### STEP 2 ###################################
#           extract cortical and subcortical values  			     #
######################################################################

# extract DK atlas information
hemispheres=("lh" "rh")
measures=("area" "volume" "thickness")
parcellations=("aparc" "aparc.a2009s")
for hemi in "${hemispheres[@]}"; do
    for meas in "${measures[@]}"; do
        for parc in "${parcellations[@]}"; do
            # Construct the table file name
            if [[ $parc == "aparc" ]]; then
                tablefile="tables/${parc}_${meas}_${hemi}.txt"
            else
                tablefile="tables/${parc}_${meas}_${hemi}.txt"
            fi
            aparcstats2table --subjects "$SUB_ID" --hemi "$hemi" --meas "$meas" --parc="$parc" --tablefile "$tablefile"
        done
    done
done

# extract segmentation information
asegstats2table --subjects $SUB_ID --common-segs --meas volume --stats=aseg.stats --table=tables/segstats.txt
asegstats2table --subjects $SUB_ID --statsfile=brainstem.v13.stats --tablefile=tables/brainstem.txt

for hemi in "${hemispheres[@]}"; do
	statsfile="thalamic-nuclei.${hemi}.v13.T1.stats"
	tablefile="tables/thalamic-nuclei_${hemi}.txt"
	asegstats2table --subjects $SUB_ID --statsfile="$statsfile" --tablefile="$tablefile"
	
	statsfile="hipposubfields.${hemi}.T2.v22.T2.stats"
	tablefile="tables/hipposubfields_${hemi}.txt"
	asegstats2table --subjects $SUB_ID --statsfile=hipposubfields.lh.T2.v22.T2.stats --tablefile=tables/hipposubfields_lh.txt 
	
	statsfile="amygdalar-nuclei.${hemi}.T2.v22.T2.stats"
	tablefile="tables/amygdalar_${hemi}.txt"
	asegstats2table --subjects $SUB_ID --statsfile=amygdalar-nuclei.lh.T2.v22.T2.stats --tablefile=tables/amygdalar_lh.txt 
done

########################### STEP 3 ###################################
#     	         Performing Tracula analysis            			 #
######################################################################

mkdir DTI
cp RAW_DTI DTI
cd $SUBJECTS_DIR/$SUB_ID/DTI
dcm2niix -f raw_dwi -o . RAW_DTI
mrconvert raw_dwi.nii raw_dwi.mif -fslgrad raw_dwi.bvec raw_dwi.bval
dwidenoise raw_dwi.mif dwi_den.mif -noise noise.mif
mrcalc raw_dwi.mif dwi_den.mif -subtract residual.mif
mrconvert dwi_den.mif dwi_den.nii

export PATH=/Applications/ants-2.5.4/bin:$PATH
antsRegistration --version

''' 
if tracula fails, check the followings:
set doeddy = 1 # should be changed to 2
set interreg = 1 # should be changed to 5

##### change by nano in trac-proc ex file.
#chage codes from
set cmd = mri_binarize
set cmd = ($cmd --i $fsdir/mri/$segname.mgz)
set cmd = ($cmd `printf "--replaceonly-nn %s 11 " $ids`)
set cmd = ($cmd --o $labdir/anatorig/${segname}+thalnuc.nii.gz)

#to
set cmd = mri_convert
set cmd = ($cmd -i $fsdir/mri/$segname.mgz)
set cmd = ($cmd `printf "--replaceonly-nn %s 11 " $ids`)
set cmd = ($cmd -o $labdir/anatorig/${segname}+thalnuc.nii.gz)
'''

trac-all -prep -c tracula_config_file.txt
trac-all -bedp -c tracula_config_file.txt
trac-all -path -c tracula_config_file.txt

########################### STEP 4 ###################################
#     	         Creating Reports per subject            			 #
######################################################################

# qcache and stats

# visualization 1
freeview -v mri/T1.mgz -v mri/wm.mgz -v mri/brainmask.mgz -v mri/aseg.auto.mgz:colormap=lut:opacity=0.2 \
-f surf/lh.white:edgecolor=blue -f surf/lh.pial:edgecolor=red -f surf/rh.white:edgecolor=blue \
-f surf/rh.pial:edgecolor=red 


# visualization 2 (Only_T1, T1 and T2)
freeview -v mri/nu.mgz -v mri/lh.hippoAmygLabels-T1.v22.mgz:colormap=lut -v mri/rh.hippoAmygLabels-T1.v22.mgz:colormap=lut -ras -18.61 -5.37 19.23 --zoom 4 -viewport axial
freeview -v mri/nu.mgz -v mri/lh.hippoAmygLabels-T1.v22.HBT.mgz:colormap=lut -v mri/rh.hippoAmygLabels-T1.v22.HBT.mgz:colormap=lut -ras -18.61 -5.37 19.23 --zoom 4 -viewport axial
freeview -v mri/nu.mgz -v mri/lh.hippoAmygLabels-T1.v22.FS60.mgz:colormap=lut -v mri/rh.hippoAmygLabels-T1.v22.FS60.mgz:colormap=lut -ras -18.61 -5.37 19.23 --zoom 4 -viewport axial
freeview -v mri/nu.mgz -v mri/lh.hippoAmygLabels-T1.v22.CA.mgz:colormap=lut -v mri/rh.hippoAmygLabels-T1.v22.CA.mgz:colormap=lut -ras -18.61 -5.37 19.23 --zoom 4 -viewport axial


freeview -v mri/nu.mgz -v mri/lh.hippoAmygLabels-T1-T2.v22.CA.mgz:colormap=lut -v mri/rh.hippoAmygLabels-T1-T2.v22.CA.mgz:colormap=lut -ras -18.61 -5.37 19.23 --zoom 4 -viewport axial


segmentHA_T2.sh  0002  /Users/payamsadeghishabestari/Downloads/PILOT2/pi_20241211_104949_8_1_3dt2_tra_0_8.nii  T2  0  $SUBJECTS_DIR # and 1


